In [180]:
### The purpose of this section is to evaluate model performance on trained models 
## first we will generate Y predictions using all trained models 
# including all NN iterations. 
import numpy as np
import pandas as pd
import csv
from scipy.sparse import save_npz, load_npz
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, \
    f1_score, log_loss, hamming_loss

In [6]:
# Load Data

train_features = load_npz('inputs/train_cv_sparse.npz')
test_features = load_npz('inputs/val_sparse.npz')

train_y = pd.read_csv('inputs/train_cv.csv')
test_y = pd.read_csv('inputs/val_data.csv')

In [13]:
# Prediction using Log_Reg Models
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


for class_name in class_names:
    filename = 'Model_W/'+class_name+'_LR.sav'
    loaded_model = joblib.load(filename)
    mp_colname = class_name+'LR_P'
    train_y[mp_colname] = loaded_model.predict(train_features)
    test_y[mp_colname] = loaded_model.predict(test_features)

In [50]:
train_y.columns

Index(['Unnamed: 0', 'id', 'comment_text', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'toxicLR_P', 'severe_toxicLR_P',
       'obsceneLR_P', 'threatLR_P', 'insultLR_P', 'identity_hateLR_P'],
      dtype='object')

In [184]:
Metrics_Names = ['roc_auc','Accuracy', 'Precision', 'recall', 'F1', 'hamming_loss']
rows = ['train_LR', 'test_LR','GRU_Train','GRU_Test']
Metrics = pd.DataFrame(data=0.00,columns=Metrics_Names,index=rows)
Metrics.head()


Metrics.loc['train_LR','roc_auc'] = roc_auc_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'])

Metrics.loc['test_LR','roc_auc'] = roc_auc_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'])


Metrics.loc['train_LR','Accuracy'] = accuracy_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'])

Metrics.loc['test_LR','Accuracy'] = accuracy_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'])


Metrics.loc['train_LR','Precision'] = precision_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                                      average='macro')

Metrics.loc['test_LR','Precision'] = precision_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                    average='macro')

Metrics.loc['train_LR','recall'] = recall_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                                  average='macro')

Metrics.loc['test_LR','recall'] = recall_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                                 average='macro')

Metrics.loc['train_LR','F1'] = f1_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                                  average='macro')

Metrics.loc['test_LR','F1'] = f1_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'],
                                                 average='macro')

Metrics.loc['train_LR','hamming_loss'] = hamming_loss(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicLR_P':'identity_hateLR_P'])

Metrics.loc['test_LR','hamming_loss'] = hamming_loss(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicLR_P':'identity_hateLR_P'])

In [185]:
Metrics

,roc_auc,Accuracy,Precision,recall,F1,hamming_loss
train_LR,0.911366,0.962716,0.948475,0.824493,0.881685,0.007197
test_LR,0.761067,0.915779,0.718867,0.528833,0.601514,0.018946
GRU_Train,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GRU_Test,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [84]:
# Load data for NN predictions

X_train = train_y["comment_text"].fillna("fillna").values
y_train = train_y[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test_y["comment_text"].fillna("fillna").values

max_features = 30000
maxlen = 150
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

embedding_path = 'inputs/glove.840B.300d.txt'
embeddings = pd.read_table(embedding_path, sep=" ", index_col=0, header=None,
                           quoting=csv.QUOTE_NONE)
emb_mean, emb_std = np.mean(embeddings.values), np.std(embeddings.values)

print(emb_mean)

-0.00583849458846


In [85]:
from datetime import datetime
now = datetime.now()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

p = 0
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = None
    if (word in embeddings.index) == True: embedding_vector = embeddings.loc[word]
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector.as_matrix()
    p = p + 1
    if p % 3000 == 0:
        print(word)

precisely


terribly


calculation


closet


parapsychology


broadcaster


tub


moratorium


topped


In [130]:
# Define Models and load weights

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(60, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()

    return model

model = get_model()
file_path = "Model_W/glove_60_D2_LR001_F30.hdf5"
model.load_weights(filepath=file_path)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 150, 300)     9000000     input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 150, 300)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 150, 160)     182880      spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
global_ave

In [131]:
# Predictions using NN to get log loss

train_score = model.predict(x_train, verbose=1, batch_size=1000)
test_score = model.predict(x_test, verbose=1, batch_size=1000)

  1000/151592 [..............................] - ETA: 13:13

  2000/151592 [..............................] - ETA: 11:11

  3000/151592 [..............................] - ETA: 10:31

  4000/151592 [..............................] - ETA: 10:56

  5000/151592 [..............................] - ETA: 11:02

...

4000/7979 [==============>...............] - ETA: 13s

5000/7979 [=================>............] - ETA: 9s 

6000/7979 [=====================>........] - ETA: 6s

7000/7979 [=========================>....] - ETA: 3s

7979/7979 [==============================] - 26s 3ms/step


In [186]:
p = 0
for class_name in class_names:
    mp_colname = class_name+'RNN_P'
    train_y[mp_colname] = train_score[:,p]
    test_y[mp_colname] = test_score[:,p]
    p+=1

In [137]:
train_y.columns

Index(['Unnamed: 0', 'id', 'comment_text', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'toxicLR_P', 'severe_toxicLR_P',
       'obsceneLR_P', 'threatLR_P', 'insultLR_P', 'identity_hateLR_P',
       'toxicRNN_P', 'severe_toxicRNN_P', 'obsceneRNN_P', 'threatRNN_P',
       'insultRNN_P', 'identity_hateRNN_P'],
      dtype='object')

In [179]:
print(np.max(train_y['toxicLR_P']))
print(np.min(train_y['toxicLR_P']))

train_y['toxicLR_P'].head()

1
0


0    0
1    0
2    0
3    1
4    1
Name: toxicLR_P, dtype: int64

In [108]:
train_y.columns

Index(['Unnamed: 0', 'id', 'comment_text', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'toxicLR_P', 'severe_toxicLR_P',
       'obsceneLR_P', 'threatLR_P', 'insultLR_P', 'identity_hateLR_P',
       'toxicRNN_P', 'severe_toxicRNN_P', 'obsceneRNN_P', 'threatRNN_P',
       'insultRNN_P', 'identity_hateRNN_P'],
      dtype='object')

In [187]:
Metrics.loc['GRU_Train','roc_auc'] = roc_auc_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'])

Metrics.loc['GRU_Test','roc_auc'] = roc_auc_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'severe_toxicRNN_P':'identity_hateRNN_P'])

Metrics.loc['GRU_Train','Accuracy'] = accuracy_score(train_y.loc[:,'toxic':'identity_hate'], 
                                       train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round())

Metrics.loc['GRU_Test','Accuracy'] = accuracy_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round())


Metrics.loc['GRU_Train','Precision'] = precision_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                                      average='macro')

Metrics.loc['GRU_Test','Precision'] = precision_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                    average='macro')

Metrics.loc['GRU_Train','recall'] = recall_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                                  average='macro')

Metrics.loc['GRU_Test','recall'] = recall_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                                 average='macro')

Metrics.loc['GRU_Train','F1'] = f1_score(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                                  average='macro')

Metrics.loc['GRU_Test','F1'] = f1_score(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round(),
                                                 average='macro')

Metrics.loc['GRU_Train','hamming_loss'] = hamming_loss(train_y.loc[:,'toxic':'identity_hate'], 
                                    train_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round())

Metrics.loc['GRU_Test','hamming_loss'] = hamming_loss(test_y.loc[:,'toxic':'identity_hate'], 
                                    test_y.loc[:,'toxicRNN_P':'identity_hateRNN_P'].round())

In [189]:
Metrics.to_csv("Metrics.csv", index=False)

Metrics

,roc_auc,Accuracy,Precision,recall,F1,hamming_loss
train_LR,0.911366,0.962716,0.948475,0.824493,0.881685,0.007197
test_LR,0.761067,0.915779,0.718867,0.528833,0.601514,0.018946
GRU_Train,0.993706,0.940393,0.792810,0.604369,0.677614,0.012254
GRU_Test,0.961227,0.923048,0.740890,0.521261,0.596676,0.017087


In [192]:
test_y.to_csv('Prediction_Values/test_y.csv')
train_y.to_csv('Prediction_Values/train_y.csv')